In [20]:
!pip install elasticsearch
!pip install python-magic

In [3]:
!pip install gdown

## Setting up the Elasticsearch

In [4]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': '7rryNDgDS1yAM3OPC7QvQA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Setting up the documents

In [5]:
# Downloading the data
"""
The data is uploaded at a google drive directory, by running this code box, 
the data will be downloaded and you will have access to it at
(enron_short/maildir)
"""
import gdown

googleDriveURL = "https://drive.google.com/file/d/10OwgK91e0lNRsrAV31sSJu7KJUujBTnE/view?usp=sharing"
output = 'enron_short.tar.gz'
gdown.download(googleDriveURL, output, quiet=False, fuzzy=True)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=10OwgK91e0lNRsrAV31sSJu7KJUujBTnE 



In [7]:
# check if the directory exists
from os import path

zipfile_path = "file_enron_short.tar.gz"
if not path.isfile(zipfile_path):
    print("the zipfile is not here, please ensure you download it first.")

In [9]:
!tar xf file_enron_short.tar.gz

In [14]:
# check if the directory exists
from os import path

documents_path = r"enron_short/maildir"
if not path.exists(documents_path):
    print("the directory is not here, please ensure you have the documnets first.")

## Your Code:

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


ModuleNotFoundError: No module named 'nltk'

In [28]:
# b) Indexer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path
import os
from collections import defaultdict

# Tokenize a document and give out a set without stopwords
def tokenize(document): 
    #Just a list of all stop words
    stop_words = set(stopwords.words('english'))

    # Tokenize or split into a list of words
    word_tokens = word_tokenize(document);
    token_set = set(word_tokens)

    # Adds word to list as lowercase, if not in stop_words
    filtered_sentence = [ w for w in token_set if not w.lower in stop_words ]

    return filtered_sentence


# Take list of documents and gives out tokenized list with the inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    # Enumerate gives an index and the value
    for doc_id, document in enumerate(documents):
        tokens = tokenize(document)
        
        for token in tokens: 
            inverted_index[token].append(doc_id)
    return inverted_index

# Writes a file with the terms of the inverted indexes
def save_index(index, filename):
    with open(filename, 'w') as file:
        for term, postings in index.items():
            file.write(f"{term}: {', '.join(map(str, postings))}\n")

def get_content(file_path: Path):
    with file_path.open(mode='r', encoding='utf-8') as file:
        content = file.read()
        return content
                
def get_path():
    current_script_directory = os.path.dirname(os.path.abspath('elk_base_enron_template.ipynb'))
    folder_name = "DataAssignment4"
    path = os.path.join(current_script_directory, folder_name)
    return path

        
def index():
    path = documents_path
    print(path)
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = Path(os.path.join(root, file))
                content = get_content(file_path)

                # Tokenize and invert indexes
                index = build_inverted_index(content)

                # Save the indexes to a file for easy access
                save_index(index, 'output.txt')

                # Return the inverted index
                return index
            
index()

C:\Users\andri\Documents\GitHub\TDT4117_Information_Retrieval\DataAssignment4


defaultdict(list,
            {'A': [0, 1019, 1686, 2705, 3372, 4391, 5057],
             'w': [2,
              73,
              97,
              111,
              119,
              190,
              196,
              391,
              463,
              491,
              500,
              530,
              653,
              706,
              717,
              816,
              845,
              880,
              911,
              922,
              1029,
              1051,
              1099,
              1197,
              1262,
              1295,
              1356,
              1374,
              1472,
              1477,
              1644,
              1688,
              1759,
              1783,
              1797,
              1805,
              1876,
              1882,
              2077,
              2149,
              2177,
              2186,
              2216,
              2339,
              2392,
              2403,
              2502,
  

In [21]:
# Index all emails folder enron_short/maildir

import os
import magic
from pathlib import Path

index_name = "enron_short"

def get_content(file_path: Path):
    with file_path.open(mode='r', encoding='utf-8') as file:
        content = file.read()
        return content

def index_to_elasticsearch(content, doc_id):
    # print(index_name, doc_id, content)
    es.index(index=index_name, id=doc_id, document={'content': content})

def get_path():
    current_script_directory = os.path.dirname(os.path.abspath('elk_base_documents_template.ipynb'))
    folder_name = "enron_short"
    path = os.path.join(current_script_directory, folder_name)
    return path

mime = magic.Magic(mime=True)

def add_to_elastic():
    path = "enron_short/maildir"
    print(path)
    for subdir, dirs, files in os.walk(path):
        for file in files:
            file_path = Path(os.path.join(subdir, file))
            file_type = mime.from_file(file_path)
            
            if file.endswith('.txt'):
                
                print(file_path)
                content = get_content(file_path)
                # Use the file name as a unique ID for the document
                doc_id = os.path.splitext(file)[0]
                #index_to_elasticsearch(content, doc_id)

add_to_elastic()

ImportError: failed to find libmagic.  Check your installation

In [16]:
# Query for "Norwegian and University and Science and Technology” and "Norwegian University Science Technology"
index="enron_short"

resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": "Norwegian and University and Science and Technology"}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_score"])

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [enron_short]', enron_short, index_expression)